In [1]:
import re
from functools import reduce
from dataclasses import dataclass, field
from typing import Set, List

def get_coeff(eqnstr):
    if eqnstr[0] == '-': eqnstr = "- " + eqnstr[1:]
    else: eqnstr = "+ " + eqnstr
    terms = re.findall(r"([+-][^XY*]+)", eqnstr.split("<=")[0])
    terms = [t+"1" if len(t) == 2 else t for t in terms]
    terms = [int(i.replace(" ", "")) for i in terms] + [int(eqnstr.split("<= ")[-1])]
    return tuple(terms)

def get_varidx(eqnstr):
    return (*map(lambda v: int(v[2:-1]) + (160 if v[0] == "X" else 0), re.findall(r"[XY]\[\d+\]", eqnstr)),)

inputvars = [*range(160)]
eqns = [
    (
        # eqn string
        l[9:], 
        # variables used
        get_varidx(l[9:]),
        # coefficients
        get_coeff(l[9:])
        
    )
    for l in open("../dist/crackme.py").read().split("\n")
    if l[:9] == "model += "
]

In [2]:
# Build adjacency
nvars = max(max(e[1]) for e in eqns)+1
adjeqns = [[] for _ in range(nvars)]
for s,vs,cs in eqns:
    for v in vs:
        adjeqns[v].append((s,vs,cs))
adjvars = [reduce(lambda x,y: x|y, (set(vs) for s,vs,cs in a)) for a in adjeqns]

In [3]:
from functools import lru_cache
from itertools import product

def get_invariant(eqnlist, inpbits):
    
    # invariant should encode uniquely
    # 1. The eqns
    # 2. The input and output
    # 3. Be comparable
    
    eqnlist = sorted([*eqnlist], key=lambda e: e[2])
    eqninv = tuple([e[2] for e in eqnlist])
    
    # this will be reference point to define position of input
    maxeqn = eqnlist[-1]
    
    # assert maxeqn contains all variables of concern in the same order        
    r = set()
    for e in eqnlist:
        r |= set(e[1])
    assert set(maxeqn[1]) == r
    
    # Fill in gaps
    for i,e in enumerate(eqnlist[:-1]):
        necoeff = list(e[2])
        j = 0
        for x in maxeqn[1]:
            if e[1][j] != x:
                necoeff.insert(j, 0)
                continue
            j += 1
        eqnlist[i] = (e[0], maxeqn[1], tuple(necoeff))
        
    assert all(maxeqn[1] == e[1] for e in eqnlist)
    eqninv = tuple([e[2] for e in eqnlist])
    
    # maps inpbits to position in mineqn
    inpmap = tuple(sorted([maxeqn[1].index(i) for i in inpbits]))
    
    # uniquely define order of output bits in mineqn
    outmap = tuple(sorted([j for j,i in enumerate(maxeqn[1]) if i not in inpbits]))
    
    # Construct invariant
    return eqnlist, (eqninv, inpmap, outmap)

@lru_cache(maxsize=None)
def check_concrete_binding(inv):
    eqninv, inpmap, outmap = inv
    szin,szout = len(inpmap), len(outmap)
    sz = szin + szout
    for inp in product([0, 1], repeat=szin):
        # For each inp there's a unique out
        nsat = 0
        for out in product([0, 1], repeat=szout):
            pt = []; ptr1,ptr2 = 0,0
            for i in range(sz):
                if i in outmap: 
                    pt.append(out[ptr1]); ptr1 += 1
                    continue
                pt.append(inp[ptr2]); ptr2 += 1
            allsat = all(
                sum(c*p for c,p in zip(e, pt)) <= e[-1]
                for e in eqninv
            )
            nsat += allsat
            if nsat > 1: 
                break
        if nsat != 1: return False
    return True

def bind(invariant, eqnlist):
    # Binds input bits and output bits
    _, inpmap, outmap = invariant
    varidx = next(iter(eqnlist))[1]
    assert len(varidx) == len(inpmap) + len(outmap)
    inpbits = (*(varidx[i] for i in inpmap),)
    outbits = (*(varidx[i] for i in outmap),)
    return inpbits, outbits

In [4]:
@dataclass
class Binding:
    invariant: int # Index of invariant in `all_invariant` array
    inp: ...
    out: ...
        
@dataclass
class Var:
    idx:int
    binding:Binding # This var is the `output` of this binding
    parents:Set[int]
    children:Set[int]

all_invariant = []
all_bindings = []
rootidx = [*range(160)]
graph = {r: Var(r, None, set(), set()) for r in rootidx}
all_layers = [set(rootidx)]
terminal_eqns = set(eqns)
while True:
    ng = graph.copy()
    newlayer = set()
    for r,rv in graph.items():
        
        adjidx = adjvars[r]
        parentidx = set((p for p in adjidx if p in graph))
        childrenidx = adjidx - parentidx
        
        for childidx in childrenidx:
            
            soleparents = parentidx & adjvars[childidx]
            sharedeqns = reduce(
                lambda x,y: x|y, 
                (
                    set(adjeqns[i]) 
                    for i in soleparents
                )
            ) & set(adjeqns[childidx])
            try: newsharedeqns, sharedeqninv = get_invariant(sharedeqns, soleparents)
            except: continue # eh just ignore for now
            if check_concrete_binding(sharedeqninv): # Binding!
                
                # sharedeqns are used to bind new variables, aren't terminal
                terminal_eqns -= sharedeqns
                
                # Create binding
                if sharedeqninv not in all_invariant:
                    all_invariant.append(sharedeqninv)
                    invidx = len(all_invariant) - 1
                else:
                    invidx = all_invariant.index(sharedeqninv)
                inpbits, outbits = bind(sharedeqninv, newsharedeqns)
                binding = Binding(
                    invidx,
                    inpbits,
                    outbits
                )
                all_bindings.append(binding)
                
                # Save new vars into ng
                for c in outbits:
                    cv = Var(c, binding, set(inpbits), set())
                    for p in soleparents:
                        ng[p].children.add(c)
                    ng[c] = cv
                    newlayer.add(c)
                    
    if len(graph) == len(ng): break
    print(len(ng), end="\r")
    graph = ng
    all_layers.append(newlayer)

In [5]:
# Solve terminal vars

from mip import Model, CBC, BINARY

model = Model(solver_name=CBC)
terminalidx = sorted([*reduce(lambda x,y: x|y, [set(e[1]) for e in terminal_eqns])])
terminal_vars = {x: model.add_var(var_type=BINARY) for x in terminalidx}

# Last layer are all the terminal stuff
assert set(terminalidx) == all_layers[-1]

for e in terminal_eqns:
    varidx, coeff = e[1], e[2]
    model += sum(terminal_vars[x]*y for x,y in zip(varidx, coeff[:-1])) <= coeff[-1]
    
model.verbose = 0
assert model.optimize().name == "OPTIMAL"
terminal_sol = {x:int(v.x) for x,v in terminal_vars.items()}

In [6]:
code = ""
for i,layer in enumerate(all_layers):
    if i == 0: continue
    code += f"# ---- LAYER {i} ----\n\n"
    nodes = sorted([graph[j] for j in layer], key=lambda v: v.binding.invariant)
    newcode = []
    for n in nodes:
        b = n.binding
        if b.invariant == 0:
            nl = f"v_{b.out[0]} = v_{b.inp[0]} ^ v_{b.inp[1]}"
            if nl not in newcode: newcode.append(nl)
        elif b.invariant == 1:
            nl = f"{', '.join('v_%d'%o for o in b.out)} = SUB[({', '.join('v_%d'%i for i in b.inp)})]"
            if nl not in newcode: newcode.append(nl)
        else:
            raise Exception()
    code += "\n".join(newcode) + "\n\n\n"
    
for i,v in terminal_sol.items():
    code += f"assert v_{i} == {v}\n"
    
open("code.txt", "w").write(code)

165517

In [7]:
def gen_subbox(inv):
    
    eqninv, inpmap, outmap = inv
    sub = {}
    szin,szout = len(inpmap), len(outmap)
    sz = szin + szout
    for inp in product([0, 1], repeat=szin):
        # For each inp there's a unique out
        for out in product([0, 1], repeat=szout):
            pt = []; ptr1,ptr2 = 0,0
            for i in range(sz):
                if i in outmap: 
                    pt.append(out[ptr1]); ptr1 += 1
                    continue
                pt.append(inp[ptr2]); ptr2 += 1
            allsat = all(
                sum(c*p for c,p in zip(e, pt)) <= e[-1]
                for e in eqninv
            )
            if allsat: 
                sub[inp] = out
                break       
    return sub

In [8]:
all_subbox = [*map(gen_subbox, all_invariant)]
for i,subbox in enumerate(all_subbox):
    print("INVARIANT: ", i)
    print("\n".join(str(i) + " : " + str(j) for i,j in subbox.items()))
    
# INVARIANT 1 is just XOR
# INVARIANT 2 is idk take it as a subbox

INVARIANT:  0
(0, 0) : (0,)
(0, 1) : (1,)
(1, 0) : (1,)
(1, 1) : (0,)
INVARIANT:  1
(0, 0, 0, 0) : (0, 0, 1, 1)
(0, 0, 0, 1) : (1, 1, 0, 0)
(0, 0, 1, 0) : (1, 0, 0, 1)
(0, 0, 1, 1) : (0, 0, 1, 0)
(0, 1, 0, 0) : (0, 1, 1, 0)
(0, 1, 0, 1) : (1, 1, 1, 1)
(0, 1, 1, 0) : (0, 1, 0, 1)
(0, 1, 1, 1) : (1, 0, 0, 0)
(1, 0, 0, 0) : (1, 0, 1, 0)
(1, 0, 0, 1) : (0, 1, 1, 1)
(1, 0, 1, 0) : (0, 0, 0, 0)
(1, 0, 1, 1) : (1, 1, 1, 0)
(1, 1, 0, 0) : (1, 1, 0, 1)
(1, 1, 0, 1) : (0, 0, 0, 1)
(1, 1, 1, 0) : (1, 0, 1, 1)
(1, 1, 1, 1) : (0, 1, 0, 0)


In [9]:
def backslice(idx, visited=None):
    
    if visited is None: visited = set()
    if idx in visited: return set()
    visited.add(idx)
    
    # root node
    parents = graph[idx].parents
    if len(parents) == 0: return set([idx])
    
    ret = set()
    for p in parents:
        ret |= backslice(p, visited)
    
    return ret

terminal_backslice = set(tuple(sorted(x)) for x in map(backslice, terminalidx))
assert len(terminal_backslice) == 2
assert terminal_backslice == set((tuple(range(80)), tuple(range(80,160))))

In [10]:
def backslice_to_layer(idx, layeridx, visited=None):
    
    if visited is None: visited = set()
    if idx in visited: return set()
    visited.add(idx)
    
    # root node
    for l in range(layeridx+1):
        if idx in all_layers[l]: 
            return set([idx])
    
    ret = set()
    parents = graph[idx].parents
    for p in parents:
        ret |= backslice_to_layer(p, layeridx, visited)
    
    return ret

In [11]:
layer1 = [idx for idx in [i for i in all_layers[1] if max(backslice(i)) < 80]]
nodes = [(n,graph[n]) for n in layer1]
nxor = [(n,k) for n,k in nodes if len(k.binding.out) == 1]
nsub = [(n,k) for n,k in nodes if len(k.binding.out) == 4]
assert len(layer1) == len(nxor) + len(nsub)
nxor = sorted(nxor, key=lambda n: [*n[1].binding.inp][::-1])
nsub = sorted(nsub, key=lambda n: ([*n[1].binding.inp][::-1], 3-n[1].binding.out.index(n[0])))
layer1_sym =  [(n[0], f"A_{format(idx, '04d')}") for idx,n in enumerate(nxor)]
layer1_sym += [(n[0], f"B_{format(idx, '04d')}") for idx,n in enumerate(nsub)]
layer1_sym = dict(layer1_sym)

assert len(set(layer1_sym.values())) == len(layer1)

def symbolize(idx):
    if idx in all_layers[0]: return f"input_{idx}"
    if idx in all_layers[1]: return layer1_sym[idx]
    return f"V_{format(idx, '04d')}"

In [12]:
def layer_to_str(idx):
    layer = [idx for idx in [i for i in all_layers[idx] if max(backslice(i)) < 80]]
    nodes = sorted([graph[j] for j in layer], key=lambda v: v.binding.invariant)
    newcode = []
    for n in nodes:
        b = n.binding
        if b.invariant == 0:
            nl = f"{symbolize(b.out[0])} = {symbolize(b.inp[0])} ^ {symbolize(b.inp[1])}"
            if nl not in newcode: newcode.append(nl)
        elif b.invariant == 1:
            nl = f"{', '.join(symbolize(o) for o in b.out)} = SUB[({', '.join(symbolize(i) for i in b.inp)})]"
            if nl not in newcode: newcode.append(nl)
        else:
            raise Exception()
    return "\n".join(sorted(newcode, key = lambda x: (len(str(x)), x)))

In [13]:
layer2 = [idx for idx in [i for i in all_layers[2] if max(backslice(i)) < 80]]
nodes = [(n,graph[n]) for n in layer2]
nsub1 = [(n,k) for n,k in nodes if len(k.binding.out) == 4 and min(k.binding.inp) >= 160]
nsub2 = [(n,k) for n,k in nodes if len(k.binding.out) == 4 and min(k.binding.inp) < 160]
assert len(layer2) == len(nsub1) + len(nsub2)
nsub1 = sorted(nsub1, key=lambda n: ([*map(symbolize, n[1].binding.inp)][::-1], 3-n[1].binding.out.index(n[0])))
nsub2 = sorted(nsub2, key=lambda n: ([*map(symbolize, n[1].binding.inp)][::-1], 3-n[1].binding.out.index(n[0])))
layer2_sym =  [(n[0], f"C_{format(idx, '04d')}") for idx,n in enumerate(nsub1)]
layer2_sym += [(n[0], f"D_{format(idx, '04d')}") for idx,n in enumerate(nsub2)]
layer2_sym = dict(layer2_sym)

assert len(set(layer2_sym.values())) == len(layer2)

def symbolize(idx):
    if idx in all_layers[0]: return f"input_{idx}"
    if idx in all_layers[1]: return layer1_sym[idx]
    if idx in all_layers[2]: return layer2_sym[idx]
    return f"V_{format(idx, '04d')}"

In [14]:
code = ""
for l in range(len(all_layers)):
    if l == 0: continue
    code += f"# --- LAYER {l} ----\n\n"
    code += layer_to_str(l)
    code += "\n\n"
    
open("code_symbolised.txt", "w").write(code)

85981

In [15]:
# Split the layers into each subgraph
all_layers1 = [
    set(n for n in l if not (max(backslice(n))) >= 80)
    for l in all_layers
]
all_layers2 = [
    l - l1
    for l1,l in zip(all_layers1, all_layers)
]

In [16]:
def gen_brute(all_layers1, start):
    
    # Traverse down from input[64:80] to see what can be derived given them

    start6480 = [start]
    ops_start6480 = []
    known6480 = set(start6480[-1])
    for i,l in enumerate(all_layers1):
        if i == 0: continue
        pl = start6480[-1]
        ns = []
        for n in l:
            if n in known6480: continue
            node = graph[n]
            parents = node.binding.inp
            if not set(parents).issubset(pl | known6480): continue
            ns.append(n)
            inv = node.binding.invariant
            if inv == 0:
                ops_start6480.append(f"X[{n}] = X[{parents[0]}] ^ X[{parents[1]}]")
                known6480.add(n)
            elif inv == 1:
                outstr = ",".join(f"X[{i}]" for i in node.binding.out)
                instr = ",".join(f"X[{i}]" for i in node.binding.inp)
                ops_start6480.append(f"{outstr} = SUB[({instr})]")
                known6480 |= set(node.binding.out)
            else: raise Exception()
        start6480.append(set(ns))
        
    
    # Traverse up from all_layers1[-1] to see what can be derived given them

    invlayers = all_layers1[::-1]
    ops_end6480 = []
    done = set()
    for i,l in enumerate(invlayers):
        if i == len(invlayers) - 1: continue

        for n in l:
            node = graph[n]
            binding = node.binding
            if binding.invariant == 0:
                diff = set(binding.inp) - (l | known6480)
                if len(diff) == 2: continue
                if len(diff) == 0: continue
                newnode = next(iter(diff))
                knownnode = next(iter(set(binding.inp) - diff))
                if newnode in done: continue
                done.add(newnode)
                ops_end6480.append(f"X[{newnode}] = X[{n}] ^ X[{knownnode}]")
            elif binding.invariant == 1:
                if not set(binding.out).issubset(l | known6480): continue
                if len(set(binding.inp) - known6480) < 4: continue
                if set(binding.inp).issubset(done): continue
                done |= set(binding.inp)
                outstr = ",".join(f"X[{i}]" for i in node.binding.out)
                instr = ",".join(f"X[{i}]" for i in node.binding.inp)
                ops_end6480.append(f"{instr} = INVSUB[({outstr})]")
            else: raise Exception()
        
    tsol = dict([(t,terminal_sol[t]) for t in all_layers1[-1]])
        
    # generate bruteforce code
    template = open("brute-template.txt").read()
    template = template.replace("~~NVARS~~", str(nvars))
    template = template.replace("~~START~~", str(min(start)))
    template = template.replace("~~TERMINALSOL~~", str(tsol))
    template = template.replace("~~PTST~~", str(0 if min(start) == 64 else 80))
    template = template.replace("~~PTEND~~", str(80 if min(start) == 64 else 160))
    template = template.replace("~~PROPAGATEFORWARD~~", "\n".join("    " + o for o in ops_start6480))
    template = template.replace("~~PROPAGATEBACKWARD~~", "\n".join("    " + o for o in ops_end6480))
    open("brute.py", "w").write(template)
    
import subprocess
gen_brute(all_layers1, set(range(64,80)))
flag = subprocess.check_output(['pypy', 'brute.py']).strip()
gen_brute(all_layers2, set(range(144,160)))
flag += subprocess.check_output(['pypy', 'brute.py']).strip()

print("Flag: SEE{" + flag.decode() + "}")

Flag: SEE{sT1lL_Us!nG_pR3&ENt?}


# Ignore from here!

In [290]:
" ".join(map(str, backslice(4198)))

'64 77 78 79'

In [205]:
from z3 import *
def boolexpr_from_truth_table(inp, out, var):
    nv = len(inp[0])
    expr = z3.BoolVal(False)
    for i,o in zip(inp, out):
        if o == 0: continue
        expr = Or(expr, And([x if v else Not(x) for x,v in zip(var, i)]))
    return simplify(expr)

def boolexpr_from_subbox(subbox, var):
    nout = len(next(iter(subbox.values())))
    ret = []
    io = [*subbox.items()]
    i = [x for x,y in io]
    oall = [y for x,y in io]
    for oidx in range(nout):
        o = [y[oidx] for y in oall]
        ret.append(boolexpr_from_truth_table(i,o,var))
    return ret

In [304]:
inv1subbox

{(0, 0, 0, 0): (0, 0, 1, 1),
 (0, 0, 0, 1): (1, 1, 0, 0),
 (0, 0, 1, 0): (1, 0, 0, 1),
 (0, 0, 1, 1): (0, 0, 1, 0),
 (0, 1, 0, 0): (0, 1, 1, 0),
 (0, 1, 0, 1): (1, 1, 1, 1),
 (0, 1, 1, 0): (0, 1, 0, 1),
 (0, 1, 1, 1): (1, 0, 0, 0),
 (1, 0, 0, 0): (1, 0, 1, 0),
 (1, 0, 0, 1): (0, 1, 1, 1),
 (1, 0, 1, 0): (0, 0, 0, 0),
 (1, 0, 1, 1): (1, 1, 1, 0),
 (1, 1, 0, 0): (1, 1, 0, 1),
 (1, 1, 0, 1): (0, 0, 0, 1),
 (1, 1, 1, 0): (1, 0, 1, 1),
 (1, 1, 1, 1): (0, 1, 0, 0)}

In [292]:
z3vars = [*[Bool("v%d"%n) for n in range(160)], *[None for _ in range(nvars - 160)]]
inv1subbox = all_subbox[1]

for i,layer in enumerate(all_layers):
    if i == 0: continue
    nodes = [graph[j] for j in layer]
    for n in nodes:
        b = n.binding
        if b.invariant == 0:
            z3vars[b.out[0]] = Xor(z3vars[b.inp[0]], z3vars[b.inp[1]])
        elif b.invariant == 1:
            newexpr = boolexpr_from_subbox(inv1subbox, [z3vars[i] for i in b.inp])
            for e,o in zip(newexpr, b.out):
                z3vars[o] = e
        else:
            raise Exception()
    print(i, end="\r")

In [293]:
s = Solver()
for i,v in terminal_sol.items():
    s.add(z3vars[i] == Bool(bool(v)))
    
print("Checking!")
s.check()

Checking!


sat

In [299]:
m = s.model()

def bytes_to_bits(pt):
    return [*map(int, "".join(format(c, "08b") for c in pt))]

def bits_to_bytes(pt):
    return bytes([int("".join(map(str, pt[8*i:8*i+8])), 2) for i in range(len(pt)//8)])

bits_to_bytes([int(eval(str(m[v]))) for v in z3vars[:160]])

b'a\x0bj\xd8\xb7\xfe\x9f\x91:2\xc9\xb3\x13\xaf$\xff\x0c8\x0b\xda'

In [413]:
code = ""
for i,layer in enumerate(all_layers):
    if i == 0: continue
    code += f"# ---- LAYER {i} ----\n\n"
    nodes = sorted([graph[j] for j in layer], key=lambda v: v.binding.invariant)
    newcode = []
    for n in nodes:
        b = n.binding
        if b.invariant == 0:
            nl = f"v_{b.out[0]} = v_{b.inp[0]} ^ v_{b.inp[1]}"
            if nl not in newcode: newcode.append(nl)
        elif b.invariant == 1:
            nl = f"{', '.join('v_%d'%o for o in b.out)} = SUB[({', '.join('v_%d'%i for i in b.inp)})]"
            if nl not in newcode: newcode.append(nl)
        else:
            raise Exception()
    code += "\n".join(newcode) + "\n\n\n"
    
for i,v in terminal_sol.items():
    code += f"assert v_{i} == {v}\n"

In [414]:
open("log", "w").write(code)

165517

In [228]:
# Visualise first few layers
from pydot import graph_from_dot_data
from graphviz import Digraph

nlayers = 7

roots = set(range(64, 80))
cutlayers = [roots]
for l in all_layers[1:nlayers]:
    cutlayers.append(set([
        n
        for n in l
        if len(graph[n].parents & cutlayers[-1]) != 0
    ]))

vis = Digraph()
for i,l in enumerate(cutlayers):
    for n in l:
        vis.node(str(n), str(n))
    if i == 0: continue
    for n in l:
        parents = graph[n].parents
        for p in parents:
            vis.edge(str(p), str(n))
            
gvis = graph_from_dot_data(vis.source)[0]
svg = gvis.create_svg()
open("dependency.svg", "wb").write(svg)

732626

In [ ]:
# -2*X[7404] + X[1113] + 6*X[3995] - 13*X[6753] <= 20

In [252]:
graph[3995]

Var(idx=3995, binding=Binding(invariant=1, inp=(3970, 7774, 4275, 6118), out=(7247, 3995, 1856, 3709)), parents={3970, 4275, 6118, 7774}, children={2593})

In [183]:
# Build dependency graph

@dataclass
class Var:
    idx:int
    parents:Set[int] = field(default_factory = lambda:set())
    children:Set[int] = field(default_factory = lambda:set())

rootidx = [*range(160)]
graph = {r: Var(r) for r in rootidx}

while True:
    ng = graph.copy()
    for r,rv in graph.items():
        cidx = adjvars[r]
        cidx = set(c for c in cidx if c not in graph) # no circular dep
        rv.children |= cidx
        for c in cidx:
            if c in ng: cv = ng[c]
            else: cv = Var(c)
            cv.parents.add(r)
            ng[c] = cv
    if len(graph) == len(ng): break
    graph = ng

In [21]:
from pydot import graph_from_dot_data
from graphviz import Digraph

# Plot dependency graph

vis = Digraph()
stack = rootidx.copy()
visited = set()
while len(stack) != 0:
    
    r = stack.pop(0)
    if r in visited: continue
    visited.add(r)
    vis.node(str(r), str(r))
    
    childs = graph[r].children
    for c in childs:
        vis.edge(str(c), str(r))
        stack.append(c)
        
open("dependency-graph.dot", "w").write(vis.source)

469614

In [185]:
idx = 64
print(graph[idx])
idxchild = 3593
print(graph[idxchild])
parenteqns = reduce(lambda x,y: x&y, (set(adjeqns[i]) for i in graph[idxchild].parents)) & set(adjeqns[idxchild])
print("\n".join([str(len(y)) + "\t" + x for x,y,z in parenteqns]))

Var(idx=64, parents=set(), children={3593, 6922, 2832, 7188, 4374, 7704, 5657, 3356, 6943, 1312, 7455, 292, 5415, 1326, 3631, 1585, 4408, 3130, 7740, 1340, 3135, 3649, 1347, 5189, 2629, 1351, 328, 3145, 585, 3831, 4935, 2381, 6740, 2135, 4953, 3163, 5215, 1889, 4449, 4707, 7521, 4198, 4711, 3179, 6764, 623, 624, 3440, 635, 3453, 5248, 1153, 6274, 6275, 644, 4229, 6792, 3721, 1928, 5260, 5778, 3987, 1686, 1687, 1692, 6812, 422, 3497, 2730, 4779, 6572, 434, 4789, 4790, 7863, 4533, 6073, 959, 7875, 7625, 1739, 204, 2766, 214, 727, 473, 3291, 5852, 2526, 5094, 7655, 744, 4585, 5357, 750, 1783})
Var(idx=3593, parents={64, 77, 78, 79}, children={5397, 6937, 1786, 6044, 1437, 7071, 1567, 4257, 2850, 6180, 168, 6185, 3370, 6443, 1454, 2609, 7603, 5941, 2998, 2489, 6458, 3129, 1601, 5073, 3030, 2138, 7775, 2915, 356, 3174, 5998, 4720, 5108, 757, 4981, 4474, 2299, 381})
8	10*X[7465] - 12*X[3433] + 4*X[7495] + 7*X[6114] + 2*Y[64] - 6*Y[79] - Y[78] + 13*Y[77] <= 16
8	3*X[7465] + 9*X[3433] + 7*X[74

In [264]:
inv

(((-1, -1, 1, 0), (-1, 1, -1, 0), (1, -1, -1, 0), (1, 1, 1, 2)), (1,), (0, 2))

In [265]:
print("\n".join(x for x,y,z in sharedeqns))
check_concrete_binding(inv)

X[1186] + Y[149] + X[11] <= 2
-X[1186] - Y[149] + X[11] <= 0
-X[1186] + Y[149] - X[11] <= 0
X[1186] - Y[149] - X[11] <= 0


False

In [275]:
all_invariants = []

root = rootidx.copy()

nextlayer = reduce(lambda x,y: x|y, (set(graph[r].children) for r in root))
for idxchild in [7455]:
    
    idxparents = graph[idxchild].parents
    sharedeqns = reduce(lambda x,y: x&y, (set(adjeqns[i]) for i in idxparents)) & set(adjeqns[idxchild])
    
    inv = get_invariant(sharedeqns, idxparents)
    if inv not in all_invariants: all_invariants.append(inv)
        
    # TODO: Check uniqueness
    # TODO: Bind
    
    break
    

In [60]:
idx = 64
reduce(lambda x,y: x&y, [set([x for x,y in adjeqns[i]]) for i in adjvars[idx]])

set()

In [65]:
v = Var(1)
v.children.add(1)
v

Var(idx=1, parents=set(), children={1})